# AMPL Model Colaboratory Template
[![colab.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/colab.ampl.com/blob/master/template/colab.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/colab.ampl.com/blob/master/template/colab.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/colab.ampl.com/blob/master/template/colab.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/colab.ampl.com/blob/master/template/colab.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/colab.ampl.com/blob/master/template/colab.ipynb) [![Hits](https://h.ampl.com/https://github.com/ampl/colab.ampl.com/blob/master/template/colab.ipynb)](https://colab.ampl.com)

Description: Basic notebook template for the AMPL Colab repository

Tags: amplpy, template, highlights

Notebook author: Filipe Brandão <<fdabrandao@gmail.com>>

In [6]:
# Install dependencies
%pip install -q amplpy pandas

In [7]:
# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

## Use `%%ampl_eval` to evaluate AMPL commands and declarations

In [8]:
%%ampl_eval
set NUTR;
set FOOD;

param cost {FOOD} > 0;
param f_min {FOOD} >= 0;
param f_max {j in FOOD} >= f_min[j];

param n_min {NUTR} >= 0;
param n_max {i in NUTR} >= n_min[i];

param amt {NUTR,FOOD} >= 0;

var Buy {j in FOOD} >= f_min[j], <= f_max[j];

minimize Total_Cost:  sum {j in FOOD} cost[j] * Buy[j];

subject to Diet {i in NUTR}:
   n_min[i] <= sum {j in FOOD} amt[i,j] * Buy[j] <= n_max[i];

## Load data directly from Python data structures using [amplpy](https://amplpy.readthedocs.io/)

In [9]:
foods = ["BEEF", "CHK", "FISH", "HAM", "MCH", "MTL", "SPG", "TUR"]
nutrients = ["A", "C", "B1", "B2", "NA", "CAL"]
ampl.set["FOOD"] = foods
ampl.param["cost"] = [3.59, 2.59, 2.29, 2.89, 1.89, 1.99, 1.99, 2.49]
ampl.param["f_min"] = [2, 2, 2, 2, 2, 2, 2, 2]
ampl.param["f_max"] = [10, 10, 10, 10, 10, 10, 10, 10]
ampl.set["NUTR"] = nutrients
ampl.param["n_min"] = [700, 700, 700, 700, 0, 16000]
ampl.param["n_max"] = [20000, 20000, 20000, 20000, 50000, 24000]
amounts = [
    [60, 8, 8, 40, 15, 70, 25, 60],
    [20, 0, 10, 40, 35, 30, 50, 20],
    [10, 20, 15, 35, 15, 15, 25, 15],
    [15, 20, 10, 10, 15, 15, 15, 10],
    [928, 2180, 945, 278, 1182, 896, 1329, 1397],
    [295, 770, 440, 430, 315, 400, 379, 450],
]
ampl.param["amt"] = {
    (nutrient, food): amounts[i][j]
    for i, nutrient in enumerate(nutrients)
    for j, food in enumerate(foods)
}

# Solve with HiGHS

In [10]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1"
ampl.solve()

HiGHS 1.3.0: tech:outlev=1
Running HiGHS 1.3.0 [date: 2022-12-14, git hash: fbd9bd3-dirty]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
6 rows, 8 cols, 47 nonzeros
6 rows, 8 cols, 47 nonzeros
Presolve : Reductions: rows 6(-0); columns 8(-0); elements 47(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     3.9440062313e+01 Pr: 5(9372.5) 0s
          4     1.1998975894e+02 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 4
Objective value     :  1.1998975894e+02
HiGHS run time      :          0.00
HiGHS 1.3.0: optimal solution; objective 119.9897589
4 simplex iterations
0 barrier iterations
absmipgap=119.99, relmipgap=inf


# Retrieve solution as a pandas dataframe

In [11]:
ampl.var["Buy"].to_pandas()

,Buy.val
BEEF,5.226933
CHK,2.000000
FISH,2.000000
HAM,10.000000
MCH,10.000000
MTL,10.000000
SPG,9.439734
TUR,2.000000
